# Similaridade

## Dataset
Kaggle: https://www.kaggle.com/geomack/spotifyclassification

Dataset de músicas do Spotify

In [1]:
import pandas as pd
import numpy as np
musicas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Similaridade/main/Musicas.csv', index_col = 0)

In [2]:
musicas.shape

(2017, 16)

In [3]:
musicas.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
id,,,,,,,,,,,,,,,,
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [4]:
musicas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2017 entries, 0 to 2016
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      2017 non-null   float64
 1   danceability      2017 non-null   float64
 2   duration_ms       2017 non-null   int64  
 3   energy            2017 non-null   float64
 4   instrumentalness  2017 non-null   float64
 5   key               2017 non-null   int64  
 6   liveness          2017 non-null   float64
 7   loudness          2017 non-null   float64
 8   mode              2017 non-null   int64  
 9   speechiness       2017 non-null   float64
 10  tempo             2017 non-null   float64
 11  time_signature    2017 non-null   float64
 12  valence           2017 non-null   float64
 13  target            2017 non-null   int64  
 14  song_title        2017 non-null   object 
 15  artist            2017 non-null   object 
dtypes: float64(10), int64(4), object(2)
memory

## Operações Knn e Range

In [5]:
def knn_query(centroConsulta,vetCaract,k):
  aux = vetCaract.copy(deep=True)
  aux['dist'] = aux.apply(lambda x: np.linalg.norm(x-centroConsulta), axis = 1)
  aux = aux.sort_values('dist')
  print(aux.head(k))
  return aux.head(k).index

In [6]:
def range_query(centroConsulta,vetCaract,raio):
  aux = vetCaract.copy(deep=True)
  aux['dist'] = aux.apply(lambda x: 1 if np.linalg.norm(x-centroConsulta) <= raio else 0, axis = 1)
  return aux.query('dist == 1').index

## Vetor de Características

https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

In [7]:
musicas.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'target',
       'song_title', 'artist'],
      dtype='object')

In [8]:
vetCaract = musicas[['acousticness','danceability','energy','instrumentalness','liveness','speechiness','valence']]

In [9]:
vetCaract.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence
id,,,,,,,
0,0.0102,0.833,0.434,0.021900,0.1650,0.4310,0.286
1,0.1990,0.743,0.359,0.006110,0.1370,0.0794,0.588
2,0.0344,0.838,0.412,0.000234,0.1590,0.2890,0.173
3,0.6040,0.494,0.338,0.510000,0.0922,0.0261,0.230
4,0.1800,0.678,0.561,0.512000,0.4390,0.0694,0.904


## Centro de consulta

In [10]:
centroConsulta = vetCaract.iloc[1].values
centroConsulta

array([0.199  , 0.743  , 0.359  , 0.00611, 0.137  , 0.0794 , 0.588  ])

In [11]:
musicas.iloc[1][['song_title','artist']]

song_title             Redbone
artist        Childish Gambino
Name: 1, dtype: object

## Consulta 1 - Knn

In [12]:
knn1 = knn_query(centroConsulta,vetCaract,3)
knn1

      acousticness  danceability  energy  ...  speechiness  valence      dist
id                                        ...                                
1            0.199         0.743   0.359  ...       0.0794    0.588  0.000000
990          0.135         0.756   0.415  ...       0.0555    0.662  0.116338
1374         0.164         0.844   0.320  ...       0.0475    0.575  0.122764

[3 rows x 8 columns]


Int64Index([1, 990, 1374], dtype='int64', name='id')

In [13]:
musicas.query('id in @knn1')[['song_title','artist']]

,song_title,artist
id,,
1,Redbone,Childish Gambino
990,No Sleep Till Brooklyn,Beastie Boys
1374,Talk,10cm


## Consulta 2 - Range query

In [14]:
range1 = range_query(centroConsulta,vetCaract,0.12)
range1

Int64Index([1, 990], dtype='int64', name='id')

In [15]:
musicas.query('id in @range1')[['song_title','artist']]

,song_title,artist
id,,
1,Redbone,Childish Gambino
990,No Sleep Till Brooklyn,Beastie Boys


## Vetor de Característica 2

In [16]:
vetCaract2 = musicas[['danceability','energy','valence']]

## Centro de consulta - Personalizado

In [17]:
centroConsulta2 = [1,1,1]
centroConsulta2 = np.array(centroConsulta2)

In [18]:
centroConsulta3 = [0,0,0]
centroConsulta3 = np.array(centroConsulta3)

## Consulta 2 - Knn

In [19]:
knn2 = knn_query(centroConsulta2,vetCaract2,3)
knn2

      danceability  energy  valence      dist
id                                           
1900         0.926   0.916    0.903  0.148125
1586         0.890   0.891    0.947  0.163677
405          0.865   0.896    0.939  0.181003


Int64Index([1900, 1586, 405], dtype='int64', name='id')

In [20]:
musicas.query('id in @knn2')[['song_title','artist']]

,song_title,artist
id,,
405,Bustin' Loose,Chuck Brown and the Soul Searchers
1586,My Favorite Snack,Imagination Movers
1900,Hollaback Girl,Gwen Stefani


## Consulta 3 - Knn

In [21]:
knn3 = knn_query(centroConsulta3,vetCaract2,3)
knn3

      danceability  energy  valence      dist
id                                           
817          0.148  0.0310   0.0638  0.164120
1598         0.156  0.0230   0.0588  0.168293
1600         0.152  0.0626   0.0484  0.171363


Int64Index([817, 1598, 1600], dtype='int64', name='id')

In [22]:
musicas.query('id in @knn3')[['song_title','artist']]

,song_title,artist
id,,
817,"Mozart: Requiem in D Minor, K. 626: VIII. Lacr...",Nikolaus Harnoncourt
1598,"Piano Quartet in E flat, Op.47: 3. Andante can...",Robert Schumann
1600,"Trio Sonata in G Major, Wq. 144: I. Adagio",Carl Philipp Emanuel Bach


# DW + Similaridade

In [23]:
exame = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/DWmedico/master/Exam.csv')

In [24]:
paciente = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/DWmedico/master/Patient.csv')

In [25]:
hospital = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/DWmedico/master/Hospital.csv')

In [26]:
dataExame = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/DWmedico/master/ExamDate.csv')

In [27]:
descricaoExame = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/DWmedico/master/ExamDescription.csv')

In [28]:
haralick = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/DWmedico/master/Haralick.csv')

## Consulta Convencional
Quantos exames com suspeita de câncer de mama foram realizados em pacientes do sexo feminino?

In [29]:
descricaoExame.head()

,PK,Equipamento,Tecnico,Medico,Contraste,Corte,ParteCorpo,RazaoInvestigacao
0,1,Mamógrafo,Technician.........1,Doctor.........1,n,sagittal,MAMA,routine
1,2,Mamógrafo,Technician.........2,Doctor.........2,y,horizontal,MAMA,tumor suspect
2,3,Mamógrafo,Technician.........3,Doctor.........3,n,coronal,MAMA,post treatment
3,4,Mamógrafo,Technician.........4,Doctor.........4,y,not informed,MAMA,not informed
4,5,Mamógrafo,Technician.........5,Doctor.........5,n,sagittal,MAMA,other


In [30]:
descricaoFiltrada = descricaoExame.query('ParteCorpo == "MAMA" and RazaoInvestigacao == "tumor suspect"')
descricaoFiltrada.shape

(751, 8)

In [31]:
paciente.head()

,PK,Nome,Nasc,Sexo,Etnia,TipoSangue,RhSangue,Cidade,Estado,Pais
0,1,Patient .............1,1/1/1901,female,white,O,positve,sao paulo,sp,brasil
1,2,Patient .............2,2/2/1902,male,black,A,negative,campinas,sp,brasil
2,3,Patient .............3,3/3/1903,ind,brown,AB,positve,santos,sp,brasil
3,4,Patient .............4,4/4/1904,female,yellow,B,negative,santo andre,sp,brasil
4,5,Patient .............5,5/5/1905,male,not informed,O,positve,piracicaba,sp,brasil


In [32]:
pacienteFiltrado = paciente.query('Sexo == "female"')
pacienteFiltrado

,PK,Nome,Nasc,Sexo,Etnia,TipoSangue,RhSangue,Cidade,Estado,Pais
0,1,Patient .............1,1/1/1901,female,white,O,positve,sao paulo,sp,brasil
3,4,Patient .............4,4/4/1904,female,yellow,B,negative,santo andre,sp,brasil
6,7,Patient .............7,7/7/1907,female,black,AB,positve,ribeirao preto,sp,brasil
9,10,Patient .............10,10/10/1910,female,not informed,A,negative,osasco,sp,brasil
12,13,Patient .............13,13/1/1913,female,brown,O,positve,rio de janeiro,rj,brasil
...,...,...,...,...,...,...,...,...,...,...
9987,9988,Patient .............9988,27/4/1966,female,brown,B,negative,buenos aires,,argentina
9990,9991,Patient .............9991,30/7/1969,female,white,AB,positve,sao paulo,sp,brasil
9993,9994,Patient .............9994,2/10/1972,female,yellow,A,negative,santo andre,sp,brasil
9996,9997,Patient .............9997,5/1/1975,female,black,O,positve,ribeirao preto,sp,brasil


In [33]:
exame.head()

,FK_DescricaoExame,FK_DataExame,FK_Paciente,FK_Hospital,FK_Haralick,Quantidade
0,1,20100101,1,1,1,1
1,2,20100102,2,2,2,1
2,3,20100103,3,3,3,1
3,4,20100104,4,4,4,1
4,5,20100105,5,5,5,1


In [34]:
examePaciente = exame.merge(pacienteFiltrado,left_on = 'FK_Paciente',right_on = 'PK')
examePacienteDescricao = examePaciente.merge(descricaoFiltrada,left_on = 'FK_DescricaoExame',right_on = 'PK')
examePacienteDescricao['Quantidade'].sum()

250

## Consulta com Similaridade
Quantos exames com suspeita de câncer de mama são similares à determinado exame considerando pacientes do sexo feminino?

In [35]:
haralick.head()

,PK,Caract1,Caract2,Caract3,Caract4
0,1,4.361686,14.349301,13.943300,13.781088
1,2,4.139440,14.466641,14.053040,13.898384
2,3,1.554944,2.159588,1.641221,2.249076
3,4,1.532200,2.260882,1.602180,2.300917
4,5,1.667880,2.276725,1.653380,2.433057


In [36]:
haralick = haralick.set_index('PK')

In [37]:
centroConsulta = haralick.iloc[0].values
centroConsulta

array([ 4.361686, 14.349301, 13.9433  , 13.781088])

In [38]:
harRange = range_query(centroConsulta,haralick,20)

In [39]:
harRange

Int64Index([    1,     2,    12,    13,    14,    15,    16,    17,    18,
               19,
            ...
            99871, 99878, 99879, 99946, 99947, 99948, 99949, 99950, 99951,
            99952],
           dtype='int64', name='PK', length=27865)

In [40]:
haralickFiltrado = haralick.query('PK in @harRange')
haralickFiltrado.shape

(27865, 4)

In [41]:
haralickFiltrado.head()

,Caract1,Caract2,Caract3,Caract4
PK,,,,
1,4.361686,14.349301,13.943300,13.781088
2,4.139440,14.466641,14.053040,13.898384
12,1.518173,4.989150,4.386849,5.324519
13,1.501782,5.146305,4.573014,5.518607
14,1.523906,4.943938,4.456168,5.406981


In [42]:
examePacienteDescricaoHaralick = examePacienteDescricao.merge(haralickFiltrado, left_on = 'FK_Haralick', right_on = 'PK')
examePacienteDescricaoHaralick['Quantidade'].sum()

42

## Função de distância (lat,long)

https://gist.github.com/rochacbruno/2883505

In [43]:
import math

def geo_dist(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [44]:
marilia = [-22.2208,-49.9486]
marilia = np.array(marilia)

bauru = [-22.31472,-49.06056]
bauru = np.array(bauru)

sp = [-23.533773,-46.625290]
sp = np.array(sp)

In [45]:
geo_dist(marilia,sp)

370.4324263349737

## DW + Similaridade (Geo)

In [46]:
city = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/DWmedico/master/City.csv')

In [47]:
city.tail()

,PK,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
24995,24996,Battle_Lake_city,MN,686,367,319,35,43,46,38,17,71,72,73,36,35,107,83,30,48.0,537,236,301,521,245,220,89,131,682,676,2,1,3,0,0,0,3,0,0,0,...,686,686,327,163,165,138,17,7,14,8,0,0,0,327,193,76,163,56,22,13,134,128,86,80,161,2.10,2.79,407,327,80,61,1.2,5.3,327,256,71,2.25,1.54,46.2836,-95.7134
24996,24997,Red_Lake_CDP,MN,1430,737,693,180,156,192,163,110,180,164,139,52,32,40,17,5,21.1,800,366,434,717,78,62,27,35,1428,26,4,1397,0,0,0,0,0,0,0,0,...,1430,1413,400,98,583,442,242,170,90,51,17,0,17,400,320,196,98,52,168,115,80,73,17,258,54,3.53,3.88,421,400,21,9,0.0,2.1,400,215,185,3.40,3.68,47.8773,-95.0121
24997,24998,Hallsville_city,MO,978,527,451,86,90,76,50,46,160,141,145,42,37,53,42,10,34.1,688,301,387,672,127,105,38,67,967,943,15,6,0,0,0,0,0,0,0,0,...,978,978,404,213,312,277,19,6,30,14,0,0,0,404,275,152,213,103,50,40,129,113,50,161,84,2.42,2.98,439,404,35,2,2.2,12.1,404,310,94,2.42,2.43,39.1170,-92.2206
24998,24999,Harrington_city,DE,3174,1719,1455,282,278,293,231,196,416,464,356,112,117,226,167,36,32.4,2180,981,1199,2052,502,429,161,268,3121,2386,687,8,14,0,0,7,3,2,1,1,...,3174,3174,1223,537,1060,873,162,84,192,91,0,0,0,1223,825,454,537,246,229,170,398,336,159,498,331,2.60,3.13,1328,1223,105,5,2.8,7.0,1223,786,437,2.60,2.58,38.9244,-75.5747
24999,25000,Lake_Mills_city,WI,4843,2510,2333,350,334,404,327,221,705,797,655,202,148,341,230,129,36.0,3528,1682,1846,3399,781,700,271,429,4806,4724,8,12,42,13,10,6,1,5,0,7,...,4843,4788,1924,1078,1480,1243,88,36,218,118,55,41,14,1924,1289,650,1078,523,153,91,635,516,250,681,492,2.49,3.05,2065,1924,141,68,1.8,4.8,1924,1325,599,2.71,2.00,43.0801,-88.9092


In [48]:
exame['FK_City'] = 0

In [49]:
count = 1
for index, row in exame.iterrows():
  if (count > 25000):
    count = 1
  row['FK_City'] = count
  count = count + 1

In [50]:
exame.tail()

,FK_DescricaoExame,FK_DataExame,FK_Paciente,FK_Hospital,FK_Haralick,Quantidade,FK_City
99995,99996,20191004,9996,21,99996,1,24996
99996,99997,20191005,9997,22,99997,1,24997
99997,99998,20191006,9998,23,99998,1,24998
99998,99999,20191007,9999,24,99999,1,24999
99999,100000,20191008,10000,25,100000,1,25000


In [51]:
city.head()

,PK,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,1,Winder_city,GA,10201,5365,4836,814,751,712,631,746,1676,1479,1155,421,371,661,536,248,33.6,7519,3430,4089,7135,1668,1445,456,989,10013,7846,1838,26,125,23,13,5,1,1,8,74,...,10201,9809,3877,1771,3019,2360,664,274,478,209,392,269,123,3877,2631,1325,1771,849,677,390,1246,1060,472,1472,981,2.53,3.07,4098,3877,211,11,2.0,4.1,3877,2185,1692,2.56,2.49,33.9965,-83.7209
1,2,Colonie_village,NY,7916,4092,3824,401,487,508,491,352,958,1229,1296,427,401,835,425,106,41.2,6208,2934,3274,5949,1593,1366,601,765,7824,7257,280,7,244,52,135,17,9,5,11,15,...,7916,7901,3174,1789,2272,1559,339,119,327,177,15,0,15,3174,2255,882,1789,674,336,152,919,739,297,962,979,2.49,2.95,3264,3174,90,8,0.8,2.6,3174,2640,534,2.56,2.12,42.7209,-73.8341
2,3,Washington_city,AR,148,86,62,6,7,7,5,6,12,25,25,4,13,15,13,10,49.3,126,58,68,120,47,38,16,22,148,57,91,0,0,0,0,0,0,0,0,0,...,148,148,78,29,27,17,7,5,7,2,0,0,0,78,40,11,29,4,10,6,38,35,17,13,30,1.90,2.58,93,78,15,2,3.3,9.5,78,59,19,1.93,1.79,33.7747,-93.6815
3,4,Wanchese_CDP,NC,1527,753,774,77,87,115,117,84,220,275,239,69,60,101,64,19,37.2,1169,581,588,1115,229,184,85,99,1521,1498,5,9,2,0,0,2,0,0,0,0,...,1527,1527,614,355,435,333,58,22,65,28,0,0,0,614,433,201,355,160,60,33,181,139,46,215,137,2.49,2.96,690,614,76,49,0.4,2.6,614,465,149,2.51,2.43,35.8450,-75.6395
4,5,Selawik_city,AK,772,371,401,53,50,55,56,62,171,224,167,59,24,23,8,0,36.5,758,489,269,722,43,31,22,9,926,264,14,403,221,1,0,202,0,0,18,0,...,952,612,229,104,206,174,29,17,44,28,340,0,340,229,156,90,104,47,36,31,73,59,9,96,26,2.67,3.17,282,229,53,18,0.7,6.9,229,135,94,2.86,2.40,66.5970,-160.0140


In [52]:
geo = city[['PK','99','100']]

In [53]:
geo = geo.set_index('PK')

In [54]:
geo.head()

,99,100
PK,,
1,33.9965,-83.7209
2,42.7209,-73.8341
3,33.7747,-93.6815
4,35.8450,-75.6395
5,66.5970,-160.0140


In [55]:
ny = [40.730610, -73.935242]
ny = np.array(ny)

In [56]:
def range_geo(centroConsulta,vetCaract,range):
  aux = vetCaract.copy(deep = True)
  aux['dist'] = aux.apply(lambda x: 1 if geo_dist(np.array(x),centroConsulta) <= range else 0, axis = 1)
  return aux.query('dist == 1').index

In [57]:
geoRange = range_geo(ny,geo,500)

In [58]:
geoRange.shape

(3805,)

In [59]:
cityFiltradaGeo = city.query('PK in @geoRange')
cityFiltradaGeo

,PK,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,2,Colonie_village,NY,7916,4092,3824,401,487,508,491,352,958,1229,1296,427,401,835,425,106,41.2,6208,2934,3274,5949,1593,1366,601,765,7824,7257,280,7,244,52,135,17,9,5,11,15,...,7916,7901,3174,1789,2272,1559,339,119,327,177,15,0,15,3174,2255,882,1789,674,336,152,919,739,297,962,979,2.49,2.95,3264,3174,90,8,0.8,2.6,3174,2640,534,2.56,2.12,42.7209,-73.8341
9,10,Clinton_CDP,CT,3516,1857,1659,202,237,218,201,156,516,611,492,214,151,252,205,61,38.6,2725,1269,1456,2632,609,518,214,304,3468,3344,22,9,42,11,10,0,1,3,13,4,...,3516,3515,1515,700,956,747,116,35,228,101,1,0,1,1515,896,427,700,312,150,91,619,489,184,449,391,2.32,2.98,1624,1515,109,26,1.6,5.1,1515,999,516,2.41,2.14,41.2779,-72.5305
17,18,Pomona_CDP,NJ,4019,2057,1962,310,369,354,292,190,571,831,493,143,123,176,113,54,34.0,2799,1313,1486,2665,418,343,121,222,3943,2980,323,14,482,188,90,61,0,17,64,62,...,4019,3955,1297,791,1405,1097,282,109,180,77,64,32,32,1297,1003,593,791,480,142,75,294,233,114,650,252,3.05,3.47,1357,1297,60,14,1.5,2.6,1297,1106,191,3.19,2.24,39.4673,-74.5477
23,24,Oakland_town,MD,1930,1039,891,113,94,89,112,131,220,244,260,95,92,201,181,98,43.4,1573,719,854,1492,537,480,155,325,1925,1894,14,3,11,0,5,2,3,0,0,1,...,1930,1648,787,343,401,331,40,13,77,44,282,246,36,787,448,199,343,132,77,45,339,305,147,209,252,2.09,2.75,918,787,131,18,5.5,11.6,787,428,359,2.31,1.84,39.4105,-79.4044
27,28,Kings_Park_CDP,NY,16146,8285,7861,1214,1188,1069,869,654,2234,2912,2116,926,725,1179,712,348,37.8,12071,5785,6286,11677,2644,2239,877,1362,15978,15411,136,21,305,80,86,45,16,47,0,31,...,16146,15687,5480,3568,5382,3810,785,236,472,176,459,419,40,5480,4196,1992,3568,1744,478,195,1284,1099,579,2127,1506,2.86,3.32,5574,5480,94,26,0.3,2.1,5480,4376,1104,3.12,1.84,40.8885,-73.2426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24962,24963,East_Brookfield_CDP,MA,1410,716,694,79,106,117,83,62,155,269,206,77,56,100,72,28,39.0,1050,510,540,1007,231,200,84,116,1402,1394,3,1,2,0,0,0,0,2,0,0,...,1410,1410,527,332,429,315,63,32,59,30,0,0,0,527,394,162,332,133,45,21,133,110,62,187,147,2.68,3.09,563,527,36,23,0.2,4.8,527,427,100,2.75,2.34,42.2290,-72.0508
24965,24966,Susquehanna_Trails_CDP,PA,2134,1035,1099,165,207,199,135,87,339,424,307,60,48,101,44,18,33.5,1469,755,714,1401,188,163,76,87,2121,2093,16,2,8,0,2,1,0,3,0,2,...,2134,2134,734,501,734,630,75,26,90,52,0,0,0,734,585,340,501,280,42,33,149,114,36,355,115,2.91,3.24,851,734,117,81,1.5,1.7,734,675,59,2.92,2.78,39.7611,-76.3722
24974,24975,East_Petersburg_borough,PA,4450,2275,2175,271,319,337,289,178,589,743,661,244,216,372,197,34,38.3,3333,1571,1762,3201,735,603,280,323,4400,4229,60,7,49,0,12,3,1,3,24,6,...,4450,4433,1708,1102,1356,1057,135,45,132,78,17,0,17,1708,1327,587,1102,456,163,99,381,316,110,617,415,2.60,2.95,1766,1708,58,3,1.0,6.9,1708,1440,268,2.62,2.47,40.1001,-76.3526
24994,24995,Iselin_CDP,NJ,16698,8484,8214,1022,1029,1003,936,924,2712,2944,2320,765,706,1320,874,143,37.4,13056,6365,6691,12533,2768,2337,961,1376,16317,10796,1006,20,4201,2843,454,423,11,173,41,256,...,16698,16686,6007,3694,5082,3303,1325,306,578,150,12,0,12,6007,4513,1925,3694,1667,589,192,1494,1247,514,2104,1703,2.78,3.24,6137,6007,130,10,0.5,3.3,6007,4543,1464,2.90,2.39,40.5746,-74.3168


In [60]:
harRange = range_query(centroConsulta,haralick,30)
haralickFiltrado = haralick.query('PK in @harRange')

In [61]:
examePaciente = exame.merge(pacienteFiltrado,left_on = 'FK_Paciente',right_on = 'PK')
examePacienteDescricao = examePaciente.merge(descricaoFiltrada,left_on = 'FK_DescricaoExame',right_on = 'PK')
examePacienteDescricaoHaralick = examePacienteDescricao.merge(haralickFiltrado, left_on = 'FK_Haralick', right_on = 'PK')
examePacienteDescricaoHaralickGeo = examePacienteDescricaoHaralick.merge(cityFiltradaGeo,left_on = 'FK_City',right_on = 'PK')
examePacienteDescricaoHaralickGeo['Quantidade'].sum()

29

## DW + Similaridade (Socioeconômico)

In [62]:
socio = city[['PK','3','6','7','8','9','10','11','12','13','14','15','16','17','18']]
socio['6'] = socio['6']/socio['3']
socio['7'] = socio['7']/socio['3']
socio['8'] = socio['8']/socio['3']
socio['9'] = socio['9']/socio['3']
socio['10'] = socio['10']/socio['3']
socio['11'] = socio['11']/socio['3']
socio['12'] = socio['12']/socio['3']
socio['13'] = socio['13']/socio['3']
socio['14'] = socio['14']/socio['3']
socio['15'] = socio['15']/socio['3']
socio['16'] = socio['16']/socio['3']
socio['17'] = socio['17']/socio['3']
socio['18'] = socio['18']/socio['3']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [63]:
socio.head()

,PK,3,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,10201,0.079796,0.073620,0.069797,0.061857,0.073130,0.164298,0.144986,0.113224,0.041270,0.036369,0.064798,0.052544,0.024311
1,2,7916,0.050657,0.061521,0.064174,0.062026,0.044467,0.121021,0.155255,0.163719,0.053941,0.050657,0.105483,0.053689,0.013391
2,3,148,0.040541,0.047297,0.047297,0.033784,0.040541,0.081081,0.168919,0.168919,0.027027,0.087838,0.101351,0.087838,0.067568
3,4,1527,0.050426,0.056974,0.075311,0.076621,0.055010,0.144073,0.180092,0.156516,0.045187,0.039293,0.066143,0.041912,0.012443
4,5,772,0.068653,0.064767,0.071244,0.072539,0.080311,0.221503,0.290155,0.216321,0.076425,0.031088,0.029793,0.010363,0.000000


In [64]:
socio = socio.drop(columns = '3')
socio = socio.set_index('PK')

In [65]:
city.query('PK == 9349')

,PK,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
9348,9349,New_York_city,NY,8008278,4214074,3794204,540878,561115,530816,520641,589831,1368021,1263280,1012385,369105,314349,494794,321360,121703,34.2,6068009,2803579,3264430,5744033,1117793,937857,359953,577904,7614319,3576385,2129762,41289,787047,170899,361531,54993,22636,86473,11334,79181,...,8008278,7825848,3021588,1124305,2410420,1642612,768620,242349,500915,155721,182430,75870,106560,3021588,1853223,897856,1124305,532402,576354,312600,1168365,962624,299920,1026091,712581,2.59,3.32,3200912,3021588,179324,28157,1.7,3.2,3021588,912296,2109292,2.81,2.5,40.7042,-73.9179


In [66]:
nySocio = socio.loc[9349].values
nySocio

array([0.06753986, 0.07006687, 0.06628341, 0.06501285, 0.07365266,
       0.17082586, 0.15774677, 0.12641731, 0.04609043, 0.03925301,
       0.06178532, 0.04012848, 0.01519715])

In [67]:
rangeSocio = range_query(nySocio,socio,0.05)
rangeSocio

Int64Index([    1,     8,    12,    19,    35,    37,    74,    78,    80,
               82,
            ...
            24947, 24950, 24961, 24973, 24979, 24984, 24985, 24988, 24995,
            25000],
           dtype='int64', name='PK', length=2288)

In [68]:
cityFiltradaGeoSocio = city.query('PK in @geoRange and PK in @rangeSocio')

In [69]:
examePacienteDescricaoHaralickGeoSocio = examePacienteDescricaoHaralick.merge(cityFiltradaGeoSocio,left_on = 'FK_City',right_on = 'PK')
examePacienteDescricaoHaralickGeoSocio['Quantidade'].sum()

7